### 1. Load packages

In [13]:
TOKEN = 'xxx' #боевой M13

In [15]:
from tinkoff.invest import Client
#from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX 


# with Client(TOKEN, target=INVEST_GRPC_API_SANDBOX ) as client:
#     print(client.users.get_accounts())


with Client(TOKEN) as client:
    print(client.users.get_accounts())

GetAccountsResponse(accounts=[Account(id='2169433725', type=<AccountType.ACCOUNT_TYPE_TINKOFF: 1>, name='M13', status=<AccountStatus.ACCOUNT_STATUS_OPEN: 2>, opened_date=datetime.datetime(2022, 4, 27, 0, 0, tzinfo=datetime.timezone.utc), closed_date=datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), access_level=<AccessLevel.ACCOUNT_ACCESS_LEVEL_READ_ONLY: 2>)])


In [79]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import seaborn as sns 
import datetime
import math
import gc
import time


from IPython.display import clear_output

from sklearn.metrics import roc_auc_score

import lightgbm as lgbm

import pickle
def dump_pkl(data, filename):
  with open(filename, 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load_pkl(filename):
  with open(filename, 'rb') as handle:
    data = pickle.load(handle)
  return data



### 2. Load data

In [20]:
import logging
import os

import pandas as pd

from tinkoff.invest import Client, SecurityTradingStatus
from tinkoff.invest.services import InstrumentsService
from tinkoff.invest.utils import quotation_to_decimal

from IPython.display import clear_output

In [23]:
def get_figi(ticker):
    with Client(TOKEN) as client:
        instruments: InstrumentsService = client.instruments
        tickers = []
        for method in ["shares", "bonds", "etfs", "currencies", "futures"]:
            for item in getattr(instruments, method)().instruments:
                tickers.append(
                    {
                        "name": item.name,
                        "ticker": item.ticker,
                        "class_code": item.class_code,
                        "figi": item.figi,
                        "uid": item.uid,
                        "type": method,
                        "min_price_increment": quotation_to_decimal(
                            item.min_price_increment
                        ),
                        "scale": 9 - len(str(item.min_price_increment.nano)) + 1,
                        "lot": item.lot,
                        "trading_status": str(
                            SecurityTradingStatus(item.trading_status).name
                        ),
                        "api_trade_available_flag": item.api_trade_available_flag,
                        "currency": item.currency,
                        "exchange": item.exchange,
                        "buy_available_flag": item.buy_available_flag,
                        "sell_available_flag": item.sell_available_flag,
                        "short_enabled_flag": item.short_enabled_flag,
                        "klong": quotation_to_decimal(item.klong),
                        "kshort": quotation_to_decimal(item.kshort),
                    }
                )

        tickers_df = pd.DataFrame(tickers)

        ticker_df = tickers_df[tickers_df["ticker"] == ticker]
        #return ticker_df
        if ticker_df.empty:
            print("There is no such ticker: %s", ticker)
            return

        figi = ticker_df["figi"].iloc[0]
        return figi


def get_info():
    with Client(TOKEN) as client:
        instruments: InstrumentsService = client.instruments
        tickers = []
        for method in ["shares", "bonds", "etfs", "currencies", "futures"]:
            for item in getattr(instruments, method)().instruments:
                tickers.append(
                    {
                        "name": item.name,
                        "ticker": item.ticker,
                        "class_code": item.class_code,
                        "figi": item.figi,
                        "uid": item.uid,
                        "type": method,
                        "min_price_increment": quotation_to_decimal(
                            item.min_price_increment
                        ),
                        "scale": 9 - len(str(item.min_price_increment.nano)) + 1,
                        "lot": item.lot,
                        "trading_status": str(
                            SecurityTradingStatus(item.trading_status).name
                        ),
                        "api_trade_available_flag": item.api_trade_available_flag,
                        "currency": item.currency,
                        "exchange": item.exchange,
                        "buy_available_flag": item.buy_available_flag,
                        "sell_available_flag": item.sell_available_flag,
                        "short_enabled_flag": item.short_enabled_flag,
                        "klong": quotation_to_decimal(item.klong),
                        "kshort": quotation_to_decimal(item.kshort),
                    }
                )

        tickers_df = pd.DataFrame(tickers)
    return tickers_df

In [26]:
import os
from datetime import timedelta
import datetime

from tqdm import tqdm
import numpy as np

from tinkoff.invest import CandleInterval, Client
from tinkoff.invest.utils import now

    
def get_all_candles(ticker=None, from_=None, to_=None, interval=None):
    
    figi = get_figi(ticker)
    
    with Client(TOKEN) as client:
        
        data = [elem for elem in client.get_all_candles(figi=figi, 
                                      from_=from_-timedelta(hours=3),
                                      to=to_- timedelta(hours=3),
                                     interval=interval
                                    )
               ]

    
    #open_ = np.zeros(len(data))
    #high = np.zeros(len(data))
    #low = np.zeros(len(data))
    close = np.zeros(len(data))
    volume = np.zeros(len(data))
    times = np.zeros(len(data), dtype='O')
    for i, elem in enumerate(data):
        #open_[i] = quotation_to_decimal(elem.open)
        #high[i] = quotation_to_decimal(elem.high)
        #low[i] = quotation_to_decimal(elem.low)
        close[i] = quotation_to_decimal(elem.close)
        volume[i] = elem.volume
        times[i] = elem.time


    df = pd.DataFrame({'time' : times,
                      #'open' : open_,
                      'close' : close,
                      'volume' : volume,
                      #'low' : low,
                      #'high' : high
                    })
    df['time'] += pd.Timedelta(hours=3)
    return df

### -1


In [30]:
get_figi('SFIN')

'BBG003LYCMB1'

In [32]:
ticker = 'T'
#2024-06-10 
data = get_all_candles(ticker=ticker,
                       from_=datetime.datetime(2024, 6, 10, tzinfo=datetime.timezone.utc),
                       to_=datetime.datetime(2024, 6, 12, tzinfo=datetime.timezone.utc),
                       interval=CandleInterval.CANDLE_INTERVAL_HOUR
                      )

data['ticker'] = ticker
data

,time,close,volume,ticker
0,2024-06-10 09:00:00+00:00,2974.0,1846.0,T
1,2024-06-10 10:00:00+00:00,2989.0,252336.0,T
2,2024-06-10 11:00:00+00:00,2976.0,106799.0,T
3,2024-06-10 12:00:00+00:00,2977.6,139763.0,T
4,2024-06-10 13:00:00+00:00,2973.6,64276.0,T
5,2024-06-10 14:00:00+00:00,2974.0,66350.0,T
6,2024-06-10 15:00:00+00:00,2948.6,133118.0,T
7,2024-06-10 16:00:00+00:00,2947.0,69133.0,T
8,2024-06-10 17:00:00+00:00,2930.6,86918.0,T
9,2024-06-10 18:00:00+00:00,2898.0,161966.0,T


In [34]:
data = get_all_candles(ticker=ticker,
                       from_=datetime.datetime(2024, 6, 10, tzinfo=datetime.timezone.utc),
                       to_=datetime.datetime(2024, 6, 12, tzinfo=datetime.timezone.utc),
                       interval=CandleInterval.CANDLE_INTERVAL_DAY
                      )

data['ticker'] = ticker
data

,time,close,volume,ticker
0,2024-06-10 03:00:00+00:00,2886.6,1391052.0,T
1,2024-06-11 03:00:00+00:00,2890.6,1160771.0,T


In [37]:
!mkdir data

In [69]:
!mkdir data/raw data/raw/1hour

mkdir: data/raw: File exists


# 1. Get data

In [317]:
stocks = [
    'ABIO',
 'AFKS',
 'AFLT',
 'ALRS',
 'APTK',
 'AQUA',
 'BANE',
 'BANEP',

 'BSPB',
 'CBOM',
 'CHMF',
 'ENPG',
 'FEES',
 'FESH',
 'FLOT',
 'GAZP',
 'GMKN',
 'GTRK',
 'HEAD',
 'HYDR',
 'IRAO',
 'IRKT',
 'KMAZ',
 'LENT',
 'LIFE',
  'LKOH',
 'LSRG',
 'MAGN',
 'MDMG',
 'MGNT',
 'MOEX',
 'MTLR',
 'MTLRP',
 'MTSS',
 'MVID',
 'NLMK',
 'NMTP',
 'NVTK',
 'OGKB',
 'PHOR',
'PIKK',
 'PLZL',
 'POSI',
 'RASP',
 'RENI',
 'RNFT',
 'ROSN',
 'RTKM',
 'RTKMP',
 'RUAL',
 'SBER',
 'SBERP',
 'SELG',
 'SFIN',
 'SGZH',
 'SIBN',
 'SMLT',
  'SNGS',
 'SNGSP',
 'SPBE',
 'SVAV',
 'T',
 'TATN',
 'TATNP',
 'TGKN',
 'TRMK',
 'TRNFP',
 'UNAC',
 'UPRO',
 'UWGN',
 'VKCO',
 'VSMO',
 'VTBR',
 'WUSH',
 'YDEX',


'TMOS',
    'X5',
]

start = datetime.datetime(2022, 1, 1, tzinfo=datetime.timezone.utc)
end = datetime.datetime(2025, 6, 1, tzinfo=datetime.timezone.utc)

In [319]:
len(stocks) #stocks + TMOS

77

In [323]:
#Основа

#1hour
for stock in tqdm(stocks):
    flag_data = False
    while not flag_data:
        try:
            data = get_all_candles(ticker=stock,
                       from_=start,
                       to_=end,
                       interval=CandleInterval.CANDLE_INTERVAL_HOUR
                      )
            flag_data = True
        except:
            print('Error: flag_data')
            time.sleep(5)

    clear_output()

    data['ticker'] = stock
    dump_pkl(data, f"./data/raw/1hour/{stock}.pkl")

100%|███████████████████████████████████████████████████████████████████| 77/77 [24:45<00:00, 19.29s/it]


In [287]:
#Доскачивание
stocks = ['NLMK', 'MAGN']

#1hour
for stock in tqdm(stocks):
    flag_data = False
    while not flag_data:
        try:
            data1 = load_pkl(f"./data/raw/1hour/{stock}.pkl").iloc[:-1]
            start = data1['time'].iloc[-1] + pd.Timedelta(hours=1)
            
            data2 = get_all_candles(ticker=stock,
                       from_=start,
                       to_=end,
                       interval=CandleInterval.CANDLE_INTERVAL_HOUR
                      )
            flag_data = True
        except:
            print('Error: flag_data')
            time.sleep(5)

    clear_output()

    
    data = pd.concat([data1, data2]).reset_index(drop=True)
    data['ticker'] = stock
    dump_pkl(data, f"./data/raw/1hour/{stock}.pkl")

100%|█████████████████████████████████████████████████████████████████████| 2/2 [00:03<00:00,  1.66s/it]


In [284]:
data_min = []
data_max = []
for stock in tqdm(stocks):
    df = load_pkl(f"./data/raw/1hour/{stock}.pkl")
    data_min += [df['time'].min()]
    data_max += [df['time'].max()]

pd.set_option('display.max_rows', 100)
pd.DataFrame({'ticker' : stocks,
             'data_min' : data_min,
             'data_max' : data_max,
             })

100%|█████████████████████████████████████████████████████████████████| 77/77 [00:00<00:00, 3029.00it/s]


,ticker,data_min,data_max
0,ABIO,2024-01-03 09:00:00+00:00,2025-04-09 13:00:00+00:00
1,AFKS,2024-01-02 10:00:00+00:00,2025-04-09 13:00:00+00:00
2,AFLT,2024-01-02 10:00:00+00:00,2025-04-09 13:00:00+00:00
3,ALRS,2024-01-02 10:00:00+00:00,2025-04-09 13:00:00+00:00
4,APTK,2024-01-03 09:00:00+00:00,2025-04-09 13:00:00+00:00
5,AQUA,2024-01-02 10:00:00+00:00,2025-04-09 13:00:00+00:00
6,BANE,2024-01-02 10:00:00+00:00,2025-04-09 13:00:00+00:00
7,BANEP,2024-01-02 10:00:00+00:00,2025-04-09 13:00:00+00:00
8,BSPB,2024-01-02 10:00:00+00:00,2025-04-09 13:00:00+00:00
9,CBOM,2024-01-03 09:00:00+00:00,2025-04-09 13:00:00+00:00


In [510]:
#Оставляем только акции, торгующиеся более 1.5 лет
need_days = int(365 * (5/7) * 1.5) 
need_days


391

In [512]:
#1hour
del_stocks_1hour = []

for stock in tqdm(stocks):
    df = load_pkl(f"./data/preproc/1hour/{stock}.pkl")

    if df.shape[0] < need_days*14: #14 часов в торг сессии
        del_stocks_1hour += [stock]
        

100%|███████████████████████████████████████████████████████████████████████| 77/77 [00:00<00:00, 2086.31it/s]


In [514]:
len(del_stocks_1hour)

0

In [516]:
del_stocks_1hour

[]

In [519]:
#1day
del_stocks_1day = []

for stock in tqdm(stocks):
    df = load_pkl(f"./data/preproc/1day/{stock}.pkl")

    if df.shape[0] < need_days: #14 часов в торг сессии
        del_stocks_1day += [stock]
        

100%|███████████████████████████████████████████████████████████████████████| 77/77 [00:00<00:00, 7443.39it/s]


In [521]:
len(del_stocks_1day)

0

In [523]:
del_stocks_1day

[]

In [526]:
#Union
del_stocks = set(del_stocks_1hour + del_stocks_1day)

In [528]:
len(del_stocks)

0

In [530]:
del_stocks

set()

In [532]:
#delete
for stock in del_stocks:
    os.remove(f"./data/preproc/1hour/{stock}.pkl")
    os.remove(f"./data/preproc/1day/{stock}.pkl")

In [535]:
import os

In [537]:
[elem.split('.') [0] for elem in os.listdir('./data/preproc/1day/')]

['AFLT',
 'IRAO',
 'MVID',
 'OGKB',
 'SPBE',
 'MTLRP',
 'SFIN',
 'KMAZ',
 'CBOM',
 'FEES',
 'LKOH',
 'FESH',
 'LENT',
 'MDMG',
 'VKCO',
 'UPRO',
 'SELG',
 'SGZH',
 'ENPG',
 'LSRG',
 'BANE',
 'NMTP',
 'PLZL',
 'TATN',
 'AFKS',
 'APTK',
 'GMKN',
 'LIFE',
 'RNFT',
 'YDEX',
 'UWGN',
 'TGKN',
 'HYDR',
 'ABIO',
 'WUSH',
 'RTKMP',
 'GTRK',
 'NLMK',
 'TATNP',
 'ALRS',
 'VSMO',
 'TRNFP',
 'MGNT',
 'RENI',
 'IRKT',
 'MTLR',
 'HEAD',
 'RUAL',
 'TMOS',
 'SIBN',
 'SNGS',
 'NVTK',
 'RASP',
 'VTBR',
 'PHOR',
 'T',
 'TRMK',
 'MOEX',
 'SBERP',
 'CHMF',
 'RTKM',
 'SMLT',
 'UNAC',
 'GAZP',
 'ROSN',
 'BELU',
 'FLOT',
 'PIKK',
 'SVAV',
 'POSI',
 'SNGSP',
 'MTSS',
 'MAGN',
 'SBER',
 'BANEP',
 'BSPB',
 'AQUA']